In [ ]:
# 07_GradCAM_Visualization.ipynb

# Introduction
# This notebook will implement Grad-CAM to visualize the regions of the image that the CNN models focus on for making predictions

# Importing Libraries
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2

# Implementing Grad-CAM
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]
    grads = tape.gradient(class_channel, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

# Visualizing Activation Maps
img_array = np.expand_dims(preprocess_input(load_img('path_to_image', target_size=(128, 128))), axis=0)
heatmap = make_gradcam_heatmap(img_array, cnn_model, 'conv_7b_ac')
plt.matshow(heatmap)
plt.show()

# Interpreting the Results
# Overlaying heatmaps on images
def overlay_heatmap(heatmap, img, alpha=0.4, colormap=cv2.COLORMAP_JET):
    heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), colormap)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
    output = cv2.addWeighted(img, alpha, heatmap, 1 - alpha, 0)
    return output